#### Read any input string and parse it (2 marks)

In [3]:
class LL1Parser:
    def __init__(self, grammar):
        self.grammar = grammar
        self.parsing_table = {}
        self.follow_sets = {}

    def construct_parsing_table(self):
        for non_terminal, productions in self.grammar.items():
            for production in productions:
                first_set = self.compute_first_set(production)
                for terminal in first_set:
                    if terminal != 'epsilon':
                        self.parsing_table[(non_terminal, terminal)] = production

                if 'epsilon' in first_set:
                    if non_terminal not in self.follow_sets:
                        self.follow_sets[non_terminal] = set()
                    follow_set = self.compute_follow_set(non_terminal)
                    for terminal in follow_set:
                        self.parsing_table[(non_terminal, terminal)] = production

    def compute_first_set(self, production):
        first_set = set()
        for symbol in production:
            if symbol in self.grammar:
                first_set.update(self.compute_first_set(self.grammar[symbol][0]))
                if 'epsilon' not in self.compute_first_set(self.grammar[symbol][0]):
                    break
            else:
                first_set.add(symbol)
                break
        return first_set

    def compute_follow_set(self, non_terminal):
        if non_terminal in self.follow_sets:
            return self.follow_sets[non_terminal]
        
        follow_set = set()
        
        for nt, productions in self.grammar.items():
            for production in productions:
                for i, symbol in enumerate(production):
                    if symbol == non_terminal:
                        if i < len(production) - 1:
                            next_symbol = production[i + 1]
                            first_set = self.compute_first_set(production[i + 1:])
                            follow_set.update(first_set - {'epsilon'})
                            if 'epsilon' in first_set:
                                follow_set.update(self.compute_follow_set(nt))
                        else:
                            follow_set.update(self.compute_follow_set(nt))
        
        self.follow_sets[non_terminal] = follow_set
        return follow_set

    def parse(self, input_string):
        stack = ['$', 'S']  # Stack with initial symbols '$' and the start symbol 'S'
        input_string += '$'  # Add the end marker '$' to the input string
        input_index = 0

        while stack:
            top_of_stack = stack[-1]

            if top_of_stack == input_string[input_index]:
                stack.pop()
                input_index += 1
            elif top_of_stack in self.grammar and (top_of_stack, input_string[input_index]) in self.parsing_table:
                production = self.parsing_table[(top_of_stack, input_string[input_index])]
                stack.pop()
                if production != 'epsilon':
                    stack.extend(reversed(production.split()))
            else:
                # Handle parsing error
                print("Error: Parsing failed.")
                return

        if input_index == len(input_string):
            print("Parsing successful: Input string is valid.")
        else:
            print("Error: Parsing failed.")

# Example grammar
grammar = {
    'S': ['E'],
    'E': ['T + E', 'T'],
    'T': ['int * T', 'int'],
}

ll1_parser = LL1Parser(grammar)
ll1_parser.construct_parsing_table()

# Input string to parse (read from user)
input_string = input("Enter an input string: ")

# Parse the input string
ll1_parser.parse(input_string)


Error: Parsing failed.
